### Initial package imports

In [12]:
import pandas as pd
import math
import scipy.stats as st
import numpy as np

## Initial files being read in and dataset creation

In [206]:
pd.set_option('display.float_format', '{:.6f}'.format)
# Read the data
main_df = pd.read_csv('Data/Cleaned_Indices_Assignment1.csv', sep=';')

# Read the interest rate data
#interest_rate_df = pd.read_csv('Data/ECB_Rates_2012_to_2022.csv', sep=';')
interest_rate_bond_df = pd.read_csv('Data/ECB_Data_10yr_Treasury_bond.csv', sep=',')

# Convert date columns to datetime format for proper merging
main_df['Date'] = pd.to_datetime(main_df['Date'], format='%d-%m-%Y')
#interest_rate_df['Date'] = pd.to_datetime(interest_rate_df['Date'], format='%d-%m-%Y')
#gov_bond_investment_df['Date'] = pd.to_datetime(gov_bond_investment_df['Date'], format='%Y-%m-%d')
interest_rate_bond_df['Date'] = pd.to_datetime(interest_rate_bond_df['Date'], format='%Y-%m-%d')

# Merge the dataframes on the Date column
main_df = pd.merge(main_df, interest_rate_bond_df, on='Date', how='left')
#main_df = pd.merge(main_df, gov_bond_investment_df, on='Date', how='left')

# Remove rows where the bond does not have a yield curve spot rate (Market closed?)
main_df = main_df.dropna(axis=0, subset=['Yield curve spot rate, 10-year maturity - Government bond'])

# Filter the dataframe to start from 2012-01-04
main_df = main_df[main_df['Date'] >= '2012-01-04']
main_df = main_df.reset_index(drop=True)


In [207]:
# Add a column for the interest bond value per day
days_per_annum = 365
interest_bond = 1500000

# Initialize the arrays with appropriate lengths matching the DataFrame
interest_bond_vector = np.zeros(len(main_df))
interest_bond_profit_vector = np.zeros(len(main_df))
interest_bond_loss_vector = np.zeros(len(main_df))
daily_rates = np.zeros(len(main_df))

# Set initial value
interest_bond_vector[0] = interest_bond


# Calculate bond values day by day based on the daily yield rate
for i in range(len(main_df)):
    daily_rate = ((main_df['Yield curve spot rate, 10-year maturity - Government bond'].iloc[i] / (days_per_annum)) * (7/5)) / 100
    daily_rates[i] = daily_rate
    
    if i > 0:
        previous_value = interest_bond_vector[i-1]
        current_value = previous_value * (1 + daily_rate)
        interest_bond_vector[i] = current_value
        
        # Calculate change, profit/loss and return
        change = current_value - previous_value
        interest_bond_profit_vector[i] = change
        interest_bond_loss_vector[i] = -change

# Add vectors to the dataframe
main_df['Interest_Bond'] = interest_bond_vector
main_df['Interest_Bond_Profit'] = interest_bond_profit_vector
main_df['Interest_Bond_Loss'] = interest_bond_loss_vector
main_df['Interest_Bond_daily_rate'] = daily_rates


## Portfolio details

### Instruments:
- **S&P500**
- **DAX40**
- **NIKKEI**
- **EU Government Bond (10-year maturity, AAA-rated)**

### Invested amount:
- **10,000,000 EURO**

### Period:
- **01/01/2012 - 31/12/2022**

### Weights:
- **S&P500**: 0.4  
- **DAX40**: 0.3  
- **NIKKEI**: 0.15  
- **EU Government Bond**: 0.15  

### Measures:
- **Value at Risk (VaR)**: 1, 5, 10 days  
- **Expected Shortfall (ES)**  



In [208]:
main_df


,Date,S&P500_Closing,Dax40_Closing,Nikkei_Closing,U_S&P500_Returns,U_Dax40_Returns,U_Nikkei_Returns,U_S&P500_Loss,U_Dax40_Loss,U_Nikkei_Loss,...,C_Dax40_Loss,C_Nikkei_Loss,USD/EUR,JPY/EUR,TIME PERIOD,"Yield curve spot rate, 10-year maturity - Government bond",Interest_Bond,Interest_Bond_Profit,Interest_Bond_Loss,Interest_Bond_daily_rate
0,2012-01-04,1277.300000,6111.550000,8560.110000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.772500,0.010070,04 Jan 2012,2.776691,1500000.000000,0.000000,0.000000,0.000107
1,2012-01-05,1281.060000,6095.990000,8488.710000,0.002939,-0.002549,-0.008376,-3.760000,15.560000,71.400000,...,15.560000,0.722568,0.782100,0.010120,05 Jan 2012,2.784807,1500160.221773,160.221773,-160.221773,0.000107
2,2012-01-06,1277.810000,6057.920000,8390.350000,-0.002540,-0.006265,-0.011655,3.250000,38.070000,98.360000,...,38.070000,1.004256,0.786100,0.010210,06 Jan 2012,2.788371,1500320.665733,160.443961,-160.443961,0.000107
3,2012-01-09,1280.700000,6017.230000,8390.350000,0.002259,-0.006739,0.000000,-2.890000,40.690000,0.000000,...,40.690000,0.000000,0.783300,0.010190,09 Jan 2012,2.757489,1500479.349701,158.683968,-158.683968,0.000106
4,2012-01-10,1292.080000,6162.980000,8422.260000,0.008847,0.023933,0.003796,-11.380000,-145.750000,-31.910000,...,-145.750000,-0.325163,0.782600,0.010190,10 Jan 2012,2.746027,1500637.390784,158.041083,-158.041083,0.000105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,2022-12-23,3844.820000,13940.930000,26235.250000,0.005851,0.001929,-0.010338,-22.430000,-26.860000,272.620000,...,-26.860000,1.933148,0.939700,0.007039,23 Dec 2022,2.464312,1616021.850779,152.734463,-152.734463,0.000095
2801,2022-12-27,3829.250000,13995.100000,26447.870000,-0.004058,0.003878,0.001589,15.570000,-54.170000,-42.000000,...,-54.170000,-0.295638,0.937900,0.007051,27 Dec 2022,2.501054,1616176.877110,155.026331,-155.026331,0.000096
2802,2022-12-28,3783.220000,13925.600000,26340.500000,-0.012093,-0.004978,-0.004068,46.030000,69.500000,107.370000,...,69.500000,0.752234,0.934100,0.007123,28 Dec 2022,2.522043,1616333.219428,156.342318,-156.342318,0.000097
2803,2022-12-29,3849.280000,14071.720000,26093.670000,0.017311,0.010438,-0.009415,-66.060000,-146.120000,246.830000,...,-146.120000,1.740398,NaN,NaN,29 Dec 2022,2.526705,1616489.865897,156.646469,-156.646469,0.000097


## Portfolio values 

In [209]:
# initial investment 
weigths = {
    'S&P500': 0.4,
    'DAX40': 0.3,
    'NIKKEI': 0.15,
    'EU-BOND': 0.15,
}

starting_investment = 10000000  # 10 million euros
starting_date = '2012-01-04'

# Filter the main_df for the starting date
starting_row = main_df[main_df['Date'] == starting_date]

# Extract the exchange rates for the starting date
usd_to_eur = float(starting_row['USD/EUR'].iloc[0])
jpy_to_eur = float(starting_row['JPY/EUR'].iloc[0])

# Calculate the invested amounts
invested_amount_SP500 = starting_investment * weigths['S&P500'] / usd_to_eur
invested_amount_DAX40 = starting_investment * weigths['DAX40']
invested_amount_NIKKEI = starting_investment * weigths['NIKKEI'] / jpy_to_eur
invested_amount_EU_BOND = starting_investment * weigths['EU-BOND']

invested_amounts = [
    invested_amount_SP500, #in USD
    invested_amount_DAX40, #in EUR
    invested_amount_NIKKEI, #in JPY
    invested_amount_EU_BOND #in EUR
]

print(invested_amounts)


[5177993.527508091, 3000000.0, 148957298.90764648, 1500000.0]


## Returns Portfolio 

In [210]:
#create a column for the invested amount in the SP500 and multiply by the return of the next day 

main_df['SP500_Investment'] = invested_amount_SP500 * (1 + main_df['C_S&P500_Returns'].cumprod())
main_df['DAX40_Investment'] = invested_amount_DAX40 * (1 + main_df['C_Dax40_Returns'].cumprod())
main_df['NIKKEI_Investment'] = invested_amount_NIKKEI * (1 + main_df['C_Nikkei_Returns'].cumprod())
main_df['EU_BOND_Investment'] = main_df['Interest_Bond']


In [211]:
main_df

,Date,S&P500_Closing,Dax40_Closing,Nikkei_Closing,U_S&P500_Returns,U_Dax40_Returns,U_Nikkei_Returns,U_S&P500_Loss,U_Dax40_Loss,U_Nikkei_Loss,...,TIME PERIOD,"Yield curve spot rate, 10-year maturity - Government bond",Interest_Bond,Interest_Bond_Profit,Interest_Bond_Loss,Interest_Bond_daily_rate,SP500_Investment,DAX40_Investment,NIKKEI_Investment,EU_BOND_Investment
0,2012-01-04,1277.300000,6111.550000,8560.110000,NaN,NaN,NaN,NaN,NaN,NaN,...,04 Jan 2012,2.776691,1500000.000000,0.000000,0.000000,0.000107,NaN,NaN,NaN,1500000.000000
1,2012-01-05,1281.060000,6095.990000,8488.710000,0.002939,-0.002549,-0.008376,-3.760000,15.560000,71.400000,...,05 Jan 2012,2.784807,1500160.221773,160.221773,-160.221773,0.000107,5189897.180583,2992352.262000,148944667.328699,1500160.221773
2,2012-01-06,1277.810000,6057.920000,8390.350000,-0.002540,-0.006265,-0.011655,3.250000,38.070000,98.360000,...,06 Jan 2012,2.788371,1500320.665733,160.443961,-160.443961,0.000107,5177969.757817,3000047.910562,148957300.410754,1500320.665733
3,2012-01-09,1280.700000,6017.230000,8390.350000,0.002259,-0.006739,0.000000,-2.890000,40.690000,0.000000,...,09 Jan 2012,2.757489,1500479.349701,158.683968,-158.683968,0.000106,5177993.485446,2999999.677107,148957298.907646,1500479.349701
4,2012-01-10,1292.080000,6162.980000,8422.260000,0.008847,0.023933,0.003796,-11.380000,-145.750000,-31.910000,...,10 Jan 2012,2.746027,1500637.390784,158.041083,-158.041083,0.000105,5177993.527217,2999999.992272,148957298.907646,1500637.390784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,2022-12-23,3844.820000,13940.930000,26235.250000,0.005851,0.001929,-0.010338,-22.430000,-26.860000,272.620000,...,23 Dec 2022,2.464312,1616021.850779,152.734463,-152.734463,0.000095,5177993.527508,3000000.000000,148957298.907646,1616021.850779
2801,2022-12-27,3829.250000,13995.100000,26447.870000,-0.004058,0.003878,0.001589,15.570000,-54.170000,-42.000000,...,27 Dec 2022,2.501054,1616176.877110,155.026331,-155.026331,0.000096,5177993.527508,3000000.000000,148957298.907646,1616176.877110
2802,2022-12-28,3783.220000,13925.600000,26340.500000,-0.012093,-0.004978,-0.004068,46.030000,69.500000,107.370000,...,28 Dec 2022,2.522043,1616333.219428,156.342318,-156.342318,0.000097,5177993.527508,3000000.000000,148957298.907646,1616333.219428
2803,2022-12-29,3849.280000,14071.720000,26093.670000,0.017311,0.010438,-0.009415,-66.060000,-146.120000,246.830000,...,29 Dec 2022,2.526705,1616489.865897,156.646469,-156.646469,0.000097,5177993.527508,3000000.000000,148957298.907646,1616489.865897


## Expected Shortfall (ES)

In [213]:
def ES(alpha, r= 0, s= 1, df= 0):
    """
    Purpose:
        Get the ES of the normal/student model

    Inputs:
        alpha   double, level
        r       double, expected return
        s       double, volatility
        df      (optional, default= 0/normal) double, df

    Return value:
        dES     double, ES
    """
    if (df == 0):
        VaR0= st.norm.ppf(alpha)
        ES0= st.norm.pdf(VaR0) / (1-alpha)
        ES= r + s*ES0
    else:
        dVaR0= st.t.ppf(alpha, df= df)
        ES0= st.t.pdf(dVaR0, df= df)*((df + dVaR0**2)/(df-1)) / (1-alpha)

        dS2t= df/(df-2)
        c= s / np.sqrt(dS2t)
        ES= r + c*ES0

    return ES

## Input values 

### Expected returns (daily)

In [249]:
Mu_SP500 = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_S&P500_Returns'].mean() 
Mu_DAX40= main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_Dax40_Returns'].mean() 
Mu_NIKKEI= main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_Nikkei_Returns'].mean() 
Mu_EU_BOND= main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['Interest_Bond_daily_rate'].mean() 
Mu= [Mu_SP500, Mu_DAX40, Mu_NIKKEI, Mu_EU_BOND]  
Pmu = weigths['S&P500'] * Mu_SP500 + weigths['DAX40'] * Mu_DAX40 + weigths['NIKKEI'] * Mu_NIKKEI + weigths['EU-BOND'] * Mu_EU_BOND
print(Mu)
print(Pmu)

[np.float64(0.0004334705374950923), np.float64(0.0003792732422457793), np.float64(3.917385512367491e-06), np.float64(2.460743025741382e-05)]
0.0002914489100372379


### Variances

In [221]:
## variances
var_SP500 = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_S&P500_Returns'].var()
var_DAX40 = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_Dax40_Returns'].var()
var_NIKKEI = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['C_Nikkei_Returns'].var()
var_EU_BOND = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]['Interest_Bond_daily_rate'].var()
variances = [var_SP500, var_DAX40, var_NIKKEI, var_EU_BOND] 
print(variances)

[np.float64(7.823373341137111e-05), np.float64(0.00014538113320003), np.float64(1.0313971712407492e-08), np.float64(1.2151321021366375e-09)]


### Covariance matrix 

In [228]:
# Filter the data for the relevant date range
filtered_df = main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')]

# Select the relevant columns for returns
Returns = filtered_df[['C_S&P500_Returns', 'C_Dax40_Returns', 'C_Nikkei_Returns', 'Interest_Bond_daily_rate']]

# Compute the covariance matrix
CovM = Returns.cov()

# Compute the correlation matrix
CorrM = Returns.corr()

# Print the covariance matrix
print("Covariance Matrix:")
print(CovM)

# Print the correlation matrix
print("\n")  # Add a blank line for better readability
print("Correlation Matrix:")
print(CorrM)


Covariance Matrix:
                          C_S&P500_Returns  C_Dax40_Returns  C_Nikkei_Returns  \
C_S&P500_Returns                  0.000078         0.000063          0.000000   
C_Dax40_Returns                   0.000063         0.000145          0.000000   
C_Nikkei_Returns                  0.000000         0.000000          0.000000   
Interest_Bond_daily_rate         -0.000000         0.000000          0.000000   

                          Interest_Bond_daily_rate  
C_S&P500_Returns                         -0.000000  
C_Dax40_Returns                           0.000000  
C_Nikkei_Returns                          0.000000  
Interest_Bond_daily_rate                  0.000000  


Correlation Matrix:
                          C_S&P500_Returns  C_Dax40_Returns  C_Nikkei_Returns  \
C_S&P500_Returns                  1.000000         0.589185          0.189941   
C_Dax40_Returns                   0.589185         1.000000          0.309195   
C_Nikkei_Returns                  0.189941   

### Portfolio variance/standard deviation 

In [225]:
# Convert weights to a numpy array
weights = np.array([weigths['S&P500'], weigths['DAX40'], weigths['NIKKEI'], weigths['EU-BOND']])

# Calculate the portfolio variance
Pvar = np.dot(weights.T, np.dot(CovM.values, weights))
Pvol = np.sqrt(Pvar)

print(Pvar)
print(Pvol)

4.0737433799644616e-05
0.006382588330735785


# Value at Risk (VaR)

In [ ]:
def VaR(alpha, r= 0, s= 1, df= 0):
    """
    Purpose:
        Get the VaR of the normal model

    Inputs:
        alpha   double, level
        r       double, expected return
        s       double, volatility
        df      (optional) double, degrees of freedom for student-t

    Return value:
        dVaR    double, VaR
    """
    if (df == 0):
        dVaR0= st.norm.ppf(alpha)

        dVaR= r + s*dVaR0
    else:
        dVaR0= st.t.ppf(alpha, df= df)

        dS2t= df/(df-2)
        c= s / np.sqrt(dS2t)
        dVaR= r + c*dVaR0

    return dVaR

# Expected Shortfall (ES)

In [235]:
def ES(alpha, r= 0, s= 1, df= 0):
    """
    Purpose:
        Get the ES of the normal/student model

    Inputs:
        alpha   double, level
        r       double, expected return
        s       double, volatility
        df      (optional, default= 0/normal) double, df

    Return value:
        dES     double, ES
    """
    if (df == 0):
        dVaR0= st.norm.ppf(alpha)
        dES0= st.norm.pdf(dVaR0) / (1-alpha)
        dES= r + s*dES0
    else:
        dVaR0= st.t.ppf(alpha, df= df)
        dES0= st.t.pdf(dVaR0, df= df)*((df + dVaR0**2)/(df-1)) / (1-alpha)

        dS2t= df/(df-2)
        c= s / np.sqrt(dS2t)
        dES= r + c*dES0

    return dES

In [250]:
def main():
    # Magic numbers
    N = len(main_df[(main_df['Date'] >= '2012-01-04') & (main_df['Date'] <= '2021-12-31')])
    P = 50
    dMu = Pmu
    dSy= Pvol
    vAlpha= [.95, .99]
    iDF= 6

    # Initialisation
    vAlpha= np.array(vAlpha)

    # Estimation
    vVaR= VaR(vAlpha, dMu, dSy)
    vVaRt= VaR(vAlpha, dMu, dSy, df= iDF)
    print ('var, vart:', vVaR, vVaRt)

    vES= ES(vAlpha, dMu, dSy)
    vESt= ES(vAlpha, dMu, dSy, df= iDF)
    print ('es, est:', vES, vESt)

    vN= dMu+dSy*st.norm.rvs(size= 10000000)
    print (f'Simulating Norm, mean= {vN.mean()}, std= {vN.std()}')

    dVaRs= np.quantile(vN, .95)
    vI= vN >= dVaRs
    print (f'Sim, var= {dVaRs}, ES= {vN[vI].mean()}')

    dS2t= iDF/(iDF-2)
    dC= dSy / np.sqrt(dS2t)
    vT= dMu+dC*st.t.rvs(df= iDF, size= 10000000)
    print (f'Simulating t({iDF}), mean= {vT.mean()}, std= {vT.std()}')

    dVaRt= np.quantile(vT, .95)
    vI= vT >= dVaRt
    print (f'Sim, vart= {dVaRt}, ES= {vT[vI].mean()}')

    np.quantile(vT, .99)
    VaR(.95, 0, 1)


In [251]:
main()

var, vart: [0.01078987 0.01513957] [0.01041806 0.01666903]
es, est: [0.0134569  0.01730241] [0.01441809 0.02130641]
Simulating Norm, mean= 0.0002936167353884537, std= 0.006380417669026081
Sim, var= 0.01079162791309561, ES= 0.013454756244945298
Simulating t(6), mean= 0.00029007617080346426, std= 0.006380349420309787
Sim, vart= 0.010409728346831583, ES= 0.014404844369330245
